In [381]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
#lib
import pandas as pd
import numpy as np
from datetime import date
import plotly.graph_objects as go
import os

#cte
path='data/MINSA/datos.xlsx'

Departamentos =  ['AMAZONAS','ANCASH','APURIMAC','AREQUIPA','AYACUCHO','CAJAMARCA','CALLAO','CUSCO','HUANCAVELICA','HUANUCO','ICA','JUNIN','LA LIBERTAD','LAMBAYEQUE','LIMA','LORETO','MADRE DE DIOS','MOQUEGUA','PASCO','PIURA','PUNO','SAN MARTIN','TACNA','TUMBES','UCAYALI']

# Dates
path_dates='data/constants/date.csv'
dates = pd.read_csv(path_dates)
DATE = dates["DATE"]
#

# 2021#
path_aux1='dataframes/aux1.csv'
path_aux1 = pd.read_csv(path_aux1)
#

#setings 
pd.set_option('display.max_columns', None)

#cte
Departamentos =  ['AMAZONAS',
                  'ANCASH',
                  'APURIMAC',
                  'AREQUIPA',
                  'AYACUCHO',
                  'CAJAMARCA',
                  'CALLAO',
                  'CUSCO',
                  'EXTRANJERO',#
                  'HUANCAVELICA',
                  'HUANUCO',
                  'ICA',
                  'JUNIN',
                  'LA LIBERTAD',
                  'LAMBAYEQUE',
                  'LIMA',
                  'LORETO',
                  'MADRE DE DIOS',
                  'MOQUEGUA',
                  'PASCO',
                  'PIURA',
                  'PUNO',
                  'SAN MARTIN',
                  'SIN REGISTRO',#
                  'TACNA',
                  'TUMBES',
                  'UCAYALI']

#functions
def lectura(path):
    df = pd.read_excel(path, sheet_name='Tabla_SINADEF')
    
    rename=df.iloc[2].to_numpy()
    df=df.drop([0,1,2])
    df.columns = rename
    return(df)
    
def filtrosMuerteViolenta(year):
    aux=str(year)
    test_Filtro_MuerteViolenta = df.loc[ (df['MUERTE VIOLENTA'] == 'SIN REGISTRO') | (df['MUERTE VIOLENTA'] == 'NO SE CONOCE')]
    test_Filtro_MuerteViolenta = df.loc[(df['AÑO'] == aux)]
    test_Filtro_MuerteViolenta = pd.pivot_table(test_Filtro_MuerteViolenta,
                                                   index=['FECHA'],
                                                   columns=['DEPARTAMENTO DOMICILIO'],
                                                   aggfunc=['size']
                                                )
    
    test_Filtro_MuerteViolenta = test_Filtro_MuerteViolenta.replace(np.nan, 0)

    #eliminar el ultimo del año actual
    if year == date.today().year:   
        test_Filtro_MuerteViolenta.drop(test_Filtro_MuerteViolenta.tail(1).index,inplace=True) # drop last row
    else:
        test_Filtro_MuerteViolenta=test_Filtro_MuerteViolenta
    return(test_Filtro_MuerteViolenta)

    test_Filtro_MuerteViolenta = test_Filtro_MuerteViolenta["size"]
        
    return(test_Filtro_MuerteViolenta)

def deleteSizeWord(auxDF):
    auxDF = auxDF["size"]
    return(auxDF)

def drop_ext_singregistro(aux):
    aux=aux.drop(columns='EXTRANJERO')
    aux=aux.drop(columns='SIN REGISTRO')
    return(aux)

def Peru(aux):
    a=aux['AMAZONAS']+aux['ANCASH']+aux['APURIMAC']+aux['AREQUIPA']+aux['AYACUCHO']+aux['CAJAMARCA']+aux['CALLAO']+aux['CUSCO']+aux['HUANCAVELICA']+aux['HUANUCO']+aux['ICA']+aux['JUNIN']+aux['LA LIBERTAD']+aux['LAMBAYEQUE']+aux['LIMA']+df2021['LORETO']+aux['MADRE DE DIOS']+aux['MOQUEGUA']+aux['PASCO']+aux['PIURA']+aux['PUNO']+aux['SAN MARTIN']+df2021['TACNA']+df2021['TUMBES']+df2021['UCAYALI']+aux['SIN REGISTRO']+aux['EXTRANJERO']
    return(a)

print("ready")

df = lectura(path) #demora un monton. No ejecutar

df2019 = filtrosMuerteViolenta(year=2019)
df2020 = filtrosMuerteViolenta(year=2020)
df2021 = filtrosMuerteViolenta(year=2021)

df2019 = df2019["size"]
df2020 = df2020["size"]
df2021 = df2021["size"]

df2019_Peru = Peru(df2019)
df2020_Peru = Peru(df2020)
df2021_Peru = Peru(df2021)

os.remove("dataframes/departamentos2019.csv")
os.remove("dataframes/departamentos2020.csv")
os.remove("dataframes/departamentos2021.csv")

df2019.to_csv(r'dataframes/departamentos2019.csv', index=False)
df2020.to_csv(r'dataframes/departamentos2020.csv', index=False) 
df2021.to_csv(r'dataframes/departamentos2021.csv', index=False) 

df2019_ok = pd.read_csv('dataframes/departamentos2019.csv')
df2020_ok = pd.read_csv('dataframes/departamentos2020.csv')
df2021_ok = pd.read_csv('dataframes/departamentos2021.csv')

frames = [df2020_ok,df2021_ok]
aux = pd.concat(frames)


df2019_Peru = df2019.sum(axis = 1, skipna = True) 
aux_Peru = aux.sum(axis = 1, skipna = True) 
aux_Peru_MA = aux_Peru
n = 2

for i in range(0,27):
    df2019_ok[Departamentos[i]+"_MA"] = round(df2019_ok[Departamentos[i]].rolling(window=7).mean().shift(-3),n)
    aux[Departamentos[i]+"_MA"] = round(aux[Departamentos[i]].rolling(window=7).mean().shift(-3),n)#2020-2021
    
df2019_Peru_MA = round(df2019_Peru.rolling(window=7).mean().shift(-3),n)
aux_Peru_MA = round(aux_Peru_MA.rolling(window=7).mean().shift(-3),n)

def sinadef(num):
    a = Departamentos[num]
    fig = go.Figure()

    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux[a],
        mode='markers',
        name='2020-2021',
        text=aux[a],
            ))
    
    #Puntitos AUC_MA
    fig.add_trace(go.Scatter(
        x =  dates["DATE"],
        y=aux[a+'_MA'],
        mode='lines',
        name='2020-2021 media movil',
        text=aux[a],
        ))

    #Puntitos Medianas
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = df2019_ok[a+'_MA'],
        mode='lines',
        name='2019 media movil',
        line_color='green',        
        text=aux[a],
        ))
    
    #final configuration
    fig.update_layout(
        title = a +': Fallecidos por causas no violentas. <br>(Media móvil centrada: 7 días)',
           xaxis_tickformat = '%d %B <br> %Y',
        #xaxis_title="X Axis Title", No lo creo necesario
        yaxis_title="Fallecidos diarios",
        showlegend=True,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        )
    #fig.show()

    #os.mkdir("resultados/"+a)
    fig.write_html("resultados/"+a+".html")
    
    
for i in range(0,27):
    sinadef(i)     
print("regiones done")  

def sinadef_PERU():
    a = "PERÚ"
    fig = go.Figure()

    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux_Peru,
        mode='markers',
        name='2020-2021',
        text=aux_Peru,
            ))
    
    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux_Peru_MA,
        mode='lines',
        name='2019 media movil',
        text=aux_Peru_MA,
            ))
  
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = df2019_Peru_MA,
        mode='lines',
        name='2019 media movil',
        line_color='green',     
        text=aux_Peru_MA,
            ))
  
    #final configuration
    fig.update_layout(
        title = a +': Fallecidos por causas no violentas. <br>(Media móvil centrada: 7 días)',
           xaxis_tickformat = '%d %B <br> %Y',
        #xaxis_title="X Axis Title", No lo creo necesario
        yaxis_title="Fallecidos diarios",
        showlegend=True,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        )
    #fig.show()

    #os.mkdir("resultados/"+a)
    fig.write_html("resultados/Perú.html")
    print('Peru done')
    
sinadef_PERU()
print('end')

ready
regiones done
Peru done
end


In [7]:
def sinadef_PERU():
    a = "PERÚ"
    fig = go.Figure()

    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux_Peru,
        mode='markers',
        name='2020-2021',
        text=aux_Peru,
            ))
    
    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux_Peru_MA,
        mode='lines',
        name='2020-2021 media movil',
        text=aux_Peru_MA,
            ))
  
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = df2019_Peru_MA,
        mode='lines',
        name='2019 media movil',
        line_color='green',     
        text=aux_Peru_MA,
            ))
  
    #final configuration
    fig.update_layout(
        title = a +': Fallecidos por causas no violentas. <br>(Media móvil centrada: 7 días)',
           xaxis_tickformat = '%d %B <br> %Y',
        #xaxis_title="X Axis Title", No lo creo necesario
        yaxis_title="Fallecidos diarios",
        showlegend=True,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        )
    fig.show()

    #os.mkdir("resultados/"+a)
    fig.write_html("resultados/Perú.html")
    print('Peru done')

In [8]:
sinadef_PERU()

Peru done


Peru done
end


In [392]:
def sinadef(num):
    a = Departamentos[num]
    fig = go.Figure()

    #Puntitos 2020
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = aux[a],
        mode='markers',
        name='2020-2021',
        text=aux[a],
            ))
    
    #Puntitos AUC_MA
    fig.add_trace(go.Scatter(
        x =  dates["DATE"],
        y=aux[a+'_MA'],
        mode='lines',
        name='2020-2021 media movil',
        text=aux[a],
        ))

    #Puntitos Medianas
    fig.add_trace(go.Scatter(
        x = dates["DATE"],
        y = df2019_ok[a+'_MA'],
        mode='lines',
        name='2019 media movil',
        line_color='green',        
        text=aux[a],
        ))
    
    #final configuration
    fig.update_layout(
        title = a +': Fallecidos por causas no violentas. <br>(Media móvil centrada: 7 días)',
           xaxis_tickformat = '%d %B <br> %Y',
        #xaxis_title="X Axis Title", No lo creo necesario
        yaxis_title="Fallecidos diarios",
        showlegend=True,
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01)
        )
    #fig.show()

    #os.mkdir("resultados/"+a)
    fig.write_html("resultados/"+a+".html")
    
    
for i in range(0,27):
    sinadef(i)     
print("regiones done")  

regiones done
